# A progress bar for `backtrader`!

In this brief tutorial, I will show you how to set up a progress bar for `backtrader` so that if your backtesting archietcture takes advantage of processor-intenvi

In [1]:
import pandas as pd
import backtrader as bt
from tqdm import tqdm
import datetime
# If you're using a jupyter notebook, uncomment this next line #
from tqdm import tqdm_notebook as tqdm 

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Ever running a backtest and think, "dang, how long will this take?!"

For me, my strategies tend to involve processes or calculations that may be time intensive, and so it's great to be able to simply get an estimate of how long the test might take. 

## TL;DR

In your `def __init__(self):`, simply instantiate a tqdm progress bar, like so:

```
def __init__(self):
        self.startcash = self.broker.getvalue()
        self.iteration_progress = tqdm(desc='Progress: ', total=self.datas[0].close.buflen() - window)
```

Then, in your `def next(self):`, update the progress bar, like so: 

```
    def next(self):
        self.iteration_progress.update()
```

Note, my method _is_ flexible to multiple data sources. So, if you're trying to test your theory on multiple currency (or crypto!) pairs, it will estimate for this as well.

In [2]:
#  Set the amount of cash you'll start with, $100k sounds good right?
start_cash = 100000

# Set the total number of rows that you will test from your available data 
n_lines_to_test = 50000

# Set the window your strategy needs to work with, or the number of rows it looks at
window = 200

# Instantiate the strategy
class BigStrategy(bt.Strategy): 
    
    # Tell backtrader what to do the first time you run the script
    def __init__(self):
        
        self.startcash = self.broker.getvalue()
        self.iteration_progress = tqdm(desc='Total runs', total=self.datas[0].close.buflen())
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=21)

    # Tell backtrader what to do every iteration from the first one
    def next(self):

        self.iteration_progress.update()
        self.iteration_progress.set_description("Processing...")

        if not self.position:
            
            if self.rsi < 30:
                    self.buy(size=100)
            else:
                if self.rsi > 70:
                    self.sell(size=100)

In [3]:
## Create an instance of cerebro
cerebro = bt.Cerebro()

# Add your strategy to Cerebro
cerebro.addstrategy(BigStrategy)

# Set our desired cash start
cerebro.broker.setcash(start_cash)

In [4]:
# Add the analyzers we are interested in
cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")    
cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', timeframe=bt.TimeFrame.Minutes, compression=5)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trade_analyzer")
cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")

# Add a basic, 20% sizer that will use 20% of available cash per trade
cerebro.addsizer(bt.sizers.PercentSizer)

In [81]:
# Add the data, formatting it for backtrader
df = pd.read_csv('binance_ETHBTC_5m.csv')
df['date'] = pd.to_datetime(df['date'])
df['date'] = [x.to_pydatetime() for x in df['date']]
# df = df.tail(n_lines_to_test)
df.index = df['date'] 


data = bt.feeds.PandasData(dataname=df)

cerebro.adddata(data, name='ETH_BTC')

In [82]:
# Run the strategy!!
strategy = cerebro.run()

print("R Normalized Return 100: ", strategy[0].analyzers.returns.get_analysis()['rnorm100'])
print("SQN: ", strategy[0].analyzers.sqn.get_analysis()['sqn'])
print("n_trades completed: ", strategy[0].analyzers.sqn.get_analysis()['trades'])
print("Ending portfolio value: ", cerebro.broker.getvalue())

AttributeError: 'int' object has no attribute 'to_pydatetime'